In [ ]:
## histoic 

## function to calcualte the accessiabilty per neighbourhood


def calculate_distance_stats(network):
    all_pairs_shortest_paths = {}
    nodes_list = list(network.nodes())

    for start_node in nodes_list:
        shortest_paths = nx.single_source_dijkstra_path_length(network, start_node, weight='length')
        all_pairs_shortest_paths[start_node] = shortest_paths

    distances = [length for paths in all_pairs_shortest_paths.values() for length in paths.values()]

    mean_distance = statistics.mean(distances)
    median_distance = statistics.median(distances)
    min_distance = min(distances)
    max_distance = max(distances)
    distance_range = max_distance - min_distance

    return {
        "mean_distance": mean_distance,
        "median_distance": median_distance,
        "min_distance": min_distance,
        "max_distance": max_distance,
        "distance_range": distance_range
    }



In [ ]:



import warnings


with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    # error catch loop
    try:

        # Create an empty list to store results for each neighborhood
        results = []

        for index, row in neighbourhoods.iterrows():
            neighbourhood = neighbourhoods.loc[[index]]

            print("Starting index number", index)
            

            ### build a network surrounding the neighbourhood

            ## create a buffer around the edge of the neighbourhood to capture boundary roads

            # set crs
            neighbourhood = neighbourhood.to_crs('27700')

            # create a buffer neighbourhood
            neighbourhood_buffer = neighbourhood['geometry'].buffer(15)

            # convert back to a geodataframe (for later on)
            neighbourhood_buffer = gpd.GeoDataFrame(geometry=neighbourhood_buffer)

            # reset crs
            neighbourhood, neighbourhood_buffer = neighbourhood.to_crs('4326'), neighbourhood_buffer.to_crs('4326')


            try:
                # Attempt to retrieve graph nodes for the current neighbourhood
                streets = ox.graph_from_polygon(neighbourhood_buffer.geometry.iloc[0], network_type='drive', truncate_by_edge=True)

            except ValueError as e:
                print(f"Skipping index {index} due to: {e}")
                continue  # Move to the next iteration without interrupting the loop

            ## get street network
            #streets = ox.graph_from_polygon(neighbourhood_buffer.geometry.iloc[0], network_type='drive',  truncate_by_edge=True)

            # Convert the OSMnx graph to a GeoDataFrame of streets
            streets_edges = ox.graph_to_gdfs(streets, nodes=False, edges=True)
            streets_nodes = ox.graph_to_gdfs(streets, nodes=True, edges=False)







            ### Find only driveable nodes

            #Filter edges with desired 'highway' attributes containing any keyword in drive_highway_types
            drive_highway_types = ['tertiary', 'secondary', 'primary', 'unclassed', 'residential', 'primary_link', 'secondary_link', 'tertiary_link', 'living_street', 'service', 'road']

            # Function to check if any desired keyword is in the 'highway' attribute
            def contains_desired_highway_type(highway_attr):
                for keyword in drive_highway_types:
                    if keyword in highway_attr:
                        return True
                return False

            # Apply the function to create a boolean mask
            mask = streets_edges['highway'].apply(contains_desired_highway_type)
            driveable_edges = streets_edges[mask]

            # Extract unique node identifiers from the MultiIndex columns 'u' and 'v' of filtered edges
            u_nodes = driveable_edges.index.get_level_values('u').tolist()
            v_nodes = driveable_edges.index.get_level_values('v').tolist()
            driveable_nodes = set(u_nodes + v_nodes)

            # Filter nodes based on their connection to filtered edges
            driveable_nodes_gdf = streets_nodes[streets_nodes.index.isin(driveable_nodes)]










            ## find the boundary roads using 

            # Calculate the difference between neighbourhood_buffer and neighbourhood
            neighbourhood_difference = gpd.overlay(neighbourhood_buffer, neighbourhood, how='difference')

            # find all nodes within the buffer
            boundary_nodes = gpd.sjoin(driveable_nodes_gdf, neighbourhood_difference, how="inner", op='within')

            # Extract nodes from the 'boundary_nodes' index
            nodes_list = list(boundary_nodes.index)







            ## get mode based networks
            try:
                bike_streets = ox.graph_from_polygon(neighbourhood_buffer.geometry.iloc[0], network_type='bike', truncate_by_edge=True)
                walk_streets = ox.graph_from_polygon(neighbourhood_buffer.geometry.iloc[0], network_type='walk', truncate_by_edge=True)
                drive_streets = ox.graph_from_polygon(neighbourhood_buffer.geometry.iloc[0], network_type='drive', truncate_by_edge=True)

            except ValueError as e:
                print(f"Skipping index {index} due to: {e}")
                continue  # Move to the next iteration without interrupting the loop
            




            walk_stats = calculate_distance_stats(walk_streets)
            bike_stats = calculate_distance_stats(bike_streets)
            drive_stats = calculate_distance_stats(drive_streets)

            # Add the statistics to the GeoDataFrame
            neighbourhood['walk_mean_distance'] = walk_stats['mean_distance']
            neighbourhood['walk_median_distance'] = walk_stats['median_distance']
            neighbourhood['walk_min_distance'] = walk_stats['min_distance']
            neighbourhood['walk_max_distance'] = walk_stats['max_distance']
            neighbourhood['walk_distance_range'] = walk_stats['distance_range']

            neighbourhood['bike_mean_distance'] = bike_stats['mean_distance']
            neighbourhood['bike_median_distance'] = bike_stats['median_distance']
            neighbourhood['bike_min_distance'] = bike_stats['min_distance']
            neighbourhood['bike_max_distance'] = bike_stats['max_distance']
            neighbourhood['bike_distance_range'] = bike_stats['distance_range']

            neighbourhood['drive_mean_distance'] = drive_stats['mean_distance']
            neighbourhood['drive_median_distance'] = drive_stats['median_distance']
            neighbourhood['drive_min_distance'] = drive_stats['min_distance']
            neighbourhood['drive_max_distance'] = drive_stats['max_distance']
            neighbourhood['drive_distance_range'] = drive_stats['distance_range']



            # Plotting the first GeoDataFrame
            #ax = neighbourhood_buffer.plot(color='blue', label='GeoDataFrame 1')

            # Plotting the second GeoDataFrame on the same plot
            #streets_edges.plot(ax=ax, color='red', label='GeoDataFrame 2')

            # Add a legend
            #plt.legend()

            # Show the plot
            #plt.show()




            # Store statistics along with neighborhood ID or other identifying information
            result = {
                'neighbourhood_id': neighbourhood['ID'].iloc[0],  # Assuming you have an ID column
                'walk_mean_distance': walk_stats['mean_distance'],
                'walk_median_distance': walk_stats['median_distance'],
                
                'bike_mean_distance': bike_stats['mean_distance'],
                'bike_median_distance': bike_stats['median_distance'],

                'drive_mean_distance': drive_stats['mean_distance'],
                'drive_median_distance': drive_stats['median_distance'],
            }

            print("Polygon", index, "processed, moving to next")
            results.append(result)

        # Convert the results to a new dataframe
        results_df = pd.DataFrame(results)

    except Exception as e:
        print("Exception occurred:", e)
        print("Printing variables:")
        #print("Neighbourhood:", neighbourhood)
        #neighbourhood.plot()
        print("Neighbourhood Buffer:", neighbourhood_buffer)
        neighbourhood_buffer.plot()
        print("Streets Edges:", streets_edges)
        #streets_edges.plot()

        raise e





In [ ]:
# with the results calcaulted, we merge the results dataframe with the neighbourhoods geodataframe on the ID column
merged_df = neighbourhoods.merge(results_df, left_on="ID", right_on="neighbourhood_id", how="inner")

# Convert the resulting DataFrame back to a GeoDataFrame
neighbourhoods = gpd.GeoDataFrame(merged_df, geometry='geometry')

In [ ]:
################ TESTING

neighbourhoods['difference'] = neighbourhoods['walk_median_distance'] - neighbourhoods['drive_median_distance']

neighbourhoods['percentage_difference'] = ((neighbourhoods['walk_median_distance'] - neighbourhoods['drive_median_distance']) / neighbourhoods['drive_median_distance']) * 100



In [ ]:
m = folium.Map()  # Replace latitude and longitude with your desired center

# Create a choropleth map based on the "difference" column
folium.Choropleth(
    geo_data=neighbourhoods,
    data=neighbourhoods,
    columns=['ID', 'difference'],  # Columns used for the map (ID for identification, difference for color encoding)
    key_on='feature.properties.ID',  # Key to match the GeoJSON ID with dataframe
    fill_color='YlGnBu',  # Color scheme (adjust as needed)
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Difference (Walk Median Distance - Drive Median Distance)'
).add_to(m)

# Display the map
m

In [ ]:
# send to geopackage for GIS investigation.
geometry_column = neighbourhoods.geometry.name

# Iterate through the columns and convert them to strings

for column in neighbourhoods.columns:
    if column != geometry_column:
        neighbourhoods[column] = neighbourhoods[column].astype(str)
neighbourhoods.to_file(r'C:\Users\b8008458\Documents\scratch_space\output.gpkg', driver="GPKG")

Single area analysis


In [ ]:
## get just Heaton

neighbourhood = neighbourhoods[neighbourhoods['ID'] == 177]

neighbourhood.explore()

In [ ]:
### build a network surrounding the neighbourhood

## create a buffer around the edge of the neighbourhood to capture boundary roads

# set crs
neighbourhood = neighbourhood.to_crs('27700')

# create a buffer neighbourhood
neighbourhood_buffer = neighbourhood['geometry'].buffer(15)

# convert back to a geodataframe (for later on)
neighbourhood_buffer = gpd.GeoDataFrame(geometry=neighbourhood_buffer)

# reset crs
neighbourhood, neighbourhood_buffer = neighbourhood.to_crs('4326'), neighbourhood_buffer.to_crs('4326')


## get street network
streets = ox.graph_from_polygon(neighbourhood_buffer.geometry.iloc[0], network_type='drive')

# Convert the OSMnx graph to a GeoDataFrame of streets
streets_edges = ox.graph_to_gdfs(streets, nodes=False, edges=True)
streets_nodes = ox.graph_to_gdfs(streets, nodes=True, edges=False)

In [ ]:
### Find only driveable nodes

#Filter edges with desired 'highway' attributes containing any keyword in drive_highway_types
drive_highway_types = ['tertiary', 'secondary', 'primary', 'unclassed', 'residential', 'primary_link', 'secondary_link', 'tertiary_link', 'living_street', 'service', 'road']

# Function to check if any desired keyword is in the 'highway' attribute
def contains_desired_highway_type(highway_attr):
    for keyword in drive_highway_types:
        if keyword in highway_attr:
            return True
    return False

# Apply the function to create a boolean mask
mask = streets_edges['highway'].apply(contains_desired_highway_type)
driveable_edges = streets_edges[mask]

# Extract unique node identifiers from the MultiIndex columns 'u' and 'v' of filtered edges
u_nodes = driveable_edges.index.get_level_values('u').tolist()
v_nodes = driveable_edges.index.get_level_values('v').tolist()
driveable_nodes = set(u_nodes + v_nodes)

# Filter nodes based on their connection to filtered edges
driveable_nodes_gdf = streets_nodes[streets_nodes.index.isin(driveable_nodes)]


In [ ]:
## find the boundary roads using 

# Calculate the difference between neighbourhood_buffer and neighbourhood
neighbourhood_difference = gpd.overlay(neighbourhood_buffer, neighbourhood, how='difference')

# find all nodes within the buffer
boundary_nodes = gpd.sjoin(driveable_nodes_gdf, neighbourhood_difference, how="inner", op='within')

# Extract nodes from the 'boundary_nodes' index
nodes_list = list(boundary_nodes.index)

In [ ]:
## get mode based networks
bike_streets = ox.graph_from_polygon(neighbourhood_buffer.geometry.iloc[0], network_type='bike')
walk_streets = ox.graph_from_polygon(neighbourhood_buffer.geometry.iloc[0], network_type='walk')
drive_streets = ox.graph_from_polygon(neighbourhood_buffer.geometry.iloc[0], network_type='drive', simplify=True)

In [ ]:
walk_stats = calculate_distance_stats(walk_streets)
bike_stats = calculate_distance_stats(bike_streets)
drive_stats = calculate_distance_stats(drive_streets)

# Add the statistics to the GeoDataFrame
neighbourhood['walk_mean_distance'] = walk_stats['mean_distance']
neighbourhood['walk_median_distance'] = walk_stats['median_distance']
neighbourhood['walk_min_distance'] = walk_stats['min_distance']
neighbourhood['walk_max_distance'] = walk_stats['max_distance']
neighbourhood['walk_distance_range'] = walk_stats['distance_range']

neighbourhood['bike_mean_distance'] = bike_stats['mean_distance']
neighbourhood['bike_median_distance'] = bike_stats['median_distance']
neighbourhood['bike_min_distance'] = bike_stats['min_distance']
neighbourhood['bike_max_distance'] = bike_stats['max_distance']
neighbourhood['bike_distance_range'] = bike_stats['distance_range']

neighbourhood['drive_mean_distance'] = drive_stats['mean_distance']
neighbourhood['drive_median_distance'] = drive_stats['median_distance']
neighbourhood['drive_min_distance'] = drive_stats['min_distance']
neighbourhood['drive_max_distance'] = drive_stats['max_distance']
neighbourhood['drive_distance_range'] = drive_stats['distance_range']